In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from time import sleep
from datetime import datetime, timedelta

import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome(executable_path="./chromedriver")

In [3]:
# password Finalprojectlogin123!@#
# email ylevine@seas.upenn.edu
# username ylfz

# login process
driver.get("http://www.takethislife.com")    

username = driver.find_element_by_xpath("""//*[@id="navbar_username"]""")
password = driver.find_element_by_xpath("""//*[@id="navbar_password"]""")

username.send_keys("ylfz")
password.send_keys("Finalprojectlogin123!@#")

driver.find_element_by_xpath("""/html/body/div[1]/div/div/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[3]/input""").click()
sleep(10)

In [4]:
driver.get("https://www.takethislife.com/depression/christmas-blues-12938/")

In [9]:
depression = pd.read_csv('depression_raw_2.csv')

In [10]:
# depression= pd.DataFrame(columns=["username","text","posts","url","date_posted"])

In [12]:
htmls = []

In [13]:
total_posts = 20 * 968

In [20]:
driver.get(depression.iloc[-1]['url'])
for item in range(1,total_posts-len(depression)):
# for item in range(1,100):
    try:
        #get full page HTML
        html = driver.page_source

        #parse structure using BS
        html2=BeautifulSoup(html,'html.parser', from_encoding="utf-8")
        htmls.append(html2)

        #get initial post from forum
        text=html2.find("div", id=re.compile("^post_message")).get_text()

        #get username
        name=html2.find('a',class_='bigusername').get_text()

        #get post count section and parse posts
        q = html2.find("div", id="posts").get_text()
        p = re.search('Posts: (\d+)',q.replace(',',''))
        posts=p[0].split(' ')[-1]

        #get current page URL
        url = driver.current_url

        #get date posted
        date_raw = driver.find_element_by_xpath("""/html/body/table[2]/tbody/tr/td[1]/div/div[1]/div/div/div/table/tbody/tr[1]/td[1]""").text

        if 'today' in date_raw.lower():
            now = datetime.utcnow()
            date_raw = datetime.strftime(now, '%Y-%m-%d, %H:%M:%S')

        if 'yesterday' in date_raw.lower():
            now = datetime.utcnow() - timedelta(days=1)
            date_raw = datetime.strftime(now, '%Y-%m-%d, %H:%M:%S')
            

        #Proceed to next post
        driver.find_element_by_xpath("""/html/body/div[3]/div/div/div[1]/a""").click()
        depression.loc[len(depression)]=[name, text, posts, url, date_raw]
        sleep(.5)
        print(f'{len(depression)}/{total_posts}',end='\r')
    except Exception as e:
        driver.find_element_by_xpath("""/html/body/div[3]/div/div/div[1]/a[1]""").click()
        print(f'Skipping url {url} item number {item}')
driver.close()
depression

Skipping url https://www.takethislife.com/depression/being-depressed-3/ item number 43
Skipping url https://www.takethislife.com/depression/being-depressed-3/ item number 45
Skipping url https://www.takethislife.com/depression/being-depressed-3/ item number 47
Skipping url https://www.takethislife.com/depression/being-depressed-3/ item number 49
Skipping url https://www.takethislife.com/depression/being-depressed-3/ item number 51
Skipping url https://www.takethislife.com/depression/being-depressed-3/ item number 53
Skipping url https://www.takethislife.com/depression/being-depressed-3/ item number 55


KeyboardInterrupt: 

In [16]:
depression.iloc[-1]['url']

'https://www.takethislife.com/depression/tired-not-sleeping-avoiding-life-176/'

In [19]:
depression

,username,text,posts,url,date_posted
0,ayoubalharchal,\n\n\nDo you ever feel like everyone is watchi...,8,https://www.takethislife.com/depression/do-you...,"2022-07-19, 02:47:41"
1,ayoubalharchal,\n\n\nDo you compare yourself to other people ...,8,https://www.takethislife.com/depression/do-you...,"07-14-22, 05:15 PM"
2,ayoubalharchal,\n\n\nDo you find yourself apologizing all the...,8,https://www.takethislife.com/depression/do-you...,"07-14-22, 08:34 AM"
3,ayoubalharchal,"\n\n\nIF SO, READING & UNDERSTANDING THIS WILL...",8,https://www.takethislife.com/depression/do-you...,"07-12-22, 05:00 PM"
4,ayoubalharchal,\n\n\nAren’t you a little bit tired of people ...,8,https://www.takethislife.com/depression/aren-t...,"07-12-22, 07:56 AM"
...,...,...,...,...,...
19284,chicho,\n\n\nMy situation is my life is falling apart...,5,https://www.takethislife.com/depression/never-...,"06-28-06, 12:00 PM"
19285,irishred,"\n\n\n""my family wont suport me because I admi...",11944,https://www.takethislife.com/depression/argent...,"06-24-06, 01:23 PM"
19286,teaspoon,\n\n\ni must be loosing it.i mean turning to t...,16,https://www.takethislife.com/depression/loosin...,"06-28-06, 12:17 AM"
19287,Baily,\n\n\n :cry: \nEveryday it seems to continue.....,28,https://www.takethislife.com/depression/i-hate...,"06-27-06, 10:07 AM"


In [21]:
depression.to_csv('depression_raw_4.csv',index=None)

In [22]:
driver.get("https://www.takethislife.com/suicide/do-i-have-balls-go-through-290606/")

In [23]:
suicide = pd.DataFrame(columns=["username","text","posts","url","date_posted"])

In [24]:
total_posts = 20 * 453

In [28]:
driver.get(suicide.iloc[-1]['url'])
for item in range(1,total_posts-len(suicide)):
# for item in range(1,100):
    try:
        #get full page HTML
        html = driver.page_source

        #parse structure using BS
        html2=BeautifulSoup(html,'html.parser', from_encoding="utf-8")
        htmls.append(html2)

        #get initial post from forum
        text=html2.find("div", id=re.compile("^post_message")).get_text()

        #get username
        name=html2.find('a',class_='bigusername').get_text()

        #get post count section and parse posts
        q = html2.find("div", id="posts").get_text()
        p = re.search('Posts: (\d+)',q.replace(',',''))
        posts=p[0].split(' ')[-1]

        #get current page URL
        url = driver.current_url

        #get date posted
        date_raw = driver.find_element_by_xpath("""/html/body/table[2]/tbody/tr/td[1]/div/div[1]/div/div/div/table/tbody/tr[1]/td[1]""").text

        if 'today' in date_raw.lower():
            now = datetime.utcnow()
            date_raw = datetime.strftime(now, '%Y-%m-%d, %H:%M:%S')

        if 'yesterday' in date_raw.lower():
            now = datetime.utcnow() - timedelta(days=1)
            date_raw = datetime.strftime(now, '%Y-%m-%d, %H:%M:%S')
            

        #Proceed to next post
        driver.find_element_by_xpath("""/html/body/div[3]/div/div/div[1]/a""").click()
        suicide.loc[len(suicide)]=[name, text, posts, url, date_raw]
        sleep(.5)
        print(f'{len(suicide)}/{total_posts}',end='\r')
    except Exception as e:
        driver.find_element_by_xpath("""/html/body/div[3]/div/div/div[1]/a[1]""").click()
        suicide.loc[len(suicide)]=[name, text, posts, url, date_raw]
        sleep(.5)
        print(f'{len(suicide)}/{total_posts}',end='\r')
        
driver.close()
suicide

KeyboardInterrupt: 

In [29]:
suicide

,username,text,posts,url,date_posted
0,BigD93,\n\n\nSo as I've mentioned. I've been a member...,428,https://www.takethislife.com/suicide/do-i-have...,"05-15-22, 06:21 PM"
1,Carlos99,\n\n\nDoes anyone know the best way to kill yo...,1,https://www.takethislife.com/suicide/suicide-2...,"06-20-22, 02:07 PM"
2,Tesmeralda,"\n\n\nHey everyone, I want to kill myself. \n...",1,https://www.takethislife.com/suicide/feeling-s...,"06-09-22, 04:54 AM"
3,Mirk,\n\n\nI want to kill myself and I don't care a...,1,https://www.takethislife.com/suicide/idgaf-abt...,"05-20-22, 03:21 AM"
4,Raven'',"\n\n\nHello. \nI don't want you to worry, I do...",2,https://www.takethislife.com/suicide/should-i-...,"02-16-22, 10:42 PM"
...,...,...,...,...,...
2402,thisisthetime10,\n\n\nI just want to be dead.\n\nI thought I w...,10,https://www.takethislife.com/suicide/i-just-wa...,"09-23-12, 06:56 AM"
2403,PheonixBomb,"\n\n\nSeriously, I'm useless at socializing, u...",721,https://www.takethislife.com/suicide/i-have-no...,"09-12-12, 01:32 PM"
2404,Miles,\n\n\nI wish I could just take my exit. I am t...,24,https://www.takethislife.com/suicide/go-quietl...,"09-26-12, 10:00 PM"
2405,troublesometurtle,\n\n\nI dont think that I can take anymore of ...,37,https://www.takethislife.com/suicide/giving-up...,"09-25-12, 08:03 AM"


In [31]:
suicide.to_csv('suicide_raw_1.csv',header=None)